In [1]:
import numpy
from collections import deque
import pandas
import math
import pandas_datareader.data as web
import datetime
import requests
import requests_cache
import xlrd
import tempfile
import itertools

In [2]:
def get_fred(fred_series):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)
    
    start = datetime.datetime(1800, 1, 1)
    df = web.DataReader(fred_series, "fred", start, session=session)
    return df

# All FRED data can be found at https://fred.stlouisfed.org/series/SERIES_NAME
FRED_SERIES = [
    'M1329AUSM193NNBR', # 1920-1934: Yields on Short-Term United States Securities, Three-Six Month Treasury Notes and Certificates, Three Month Treasury Bills for United States
    'M1329BUSM193NNBR', # 1931-1969: Yields on Short-Term United States Securities, Three-Six Month Treasury Notes and Certificates, Three Month Treasury Bills for United States
    'M13009USM156NNBR', # 1914-1969: Discount Rates, Federal Reserve Bank of New York for United States
    'M13044USM156NNBR', # 1951-1967: Yields on Corporate Bonds, New Issues, Aa Rating for United States
    'M13035USM156NNBR', # 1919-1968: Yields on Corporate Bonds, Highest Rating for United States
    'M1333AUSM156NNBR', # 1919-1944: Yield on Long-Term United States Bonds for United States
    'M13058USM156NNBR', # 1942-1962: Yields on Twenty Year United States Government Bonds for United States
    'BAA', # 1919-2016
    'AAA', # 1919-2016
    'CP1M', # 1971-1997
    'CP3M', # 1971-1997
    'CP6M', # 1970-1997
    'GS1M', # 2001-: 1-Month Treasury Constant Maturity Rate
    'GS3M', # 1982- : 3-Month Treasury Constant Maturity Rate
    'GS6M', # 1982- : 6-Month Treasury Constant Maturity Rate
    'GS1', # 1953-
    'GS2', # 1976-
    'GS3', # 1953-
    'GS5', # 1953-
    'GS7', # 1969-
    'GS10', # 1953-
    'GS20', # 1953-
    'GS30', # 1977-
    'CD1M', # 1965-2013
    'CD3M', # 1964-2013
    'CD6M', # 1964-2013
    'TB1YR', # 1959- (with gaps)
    'TB4WK', # 2001-
    'TB3MS', # 1934-
    'TB6MS', # 1958-
    'MSLB20', # 1953-216 State and Local Bonds - Bond Buyer Go 20-Bond Municipal Bond Index (DISCONTINUED)
    'LTGOVTBD', # 1925-2000 Long-Term U.S. Government Securities (DISCONTINUED)
]

fred = get_fred(FRED_SERIES)
fred.head()

,M1329AUSM193NNBR,M1329BUSM193NNBR,M13009USM156NNBR,M13044USM156NNBR,M13035USM156NNBR,M1333AUSM156NNBR,M13058USM156NNBR,BAA,AAA,CP1M,...,GS30,CD1M,CD3M,CD6M,TB1YR,TB4WK,TB3MS,TB6MS,MSLB20,LTGOVTBD
DATE,,,,,,,,,,,,,,,,,,,,,
1914-11-01,NaN,NaN,5.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1914-12-01,NaN,NaN,5.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915-01-01,NaN,NaN,4.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915-02-01,NaN,NaN,4.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1915-03-01,NaN,NaN,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def shiller_date_converter(x):
    year = int(x)
    month = int(x * 100) - (year * 100)
    dx = datetime.date(year, month, 1)
    return dx

def get_shiller(url="http://www.econ.yale.edu/~shiller/data/ie_data.xls"):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    r = session.get(url, stream=True)

    with tempfile.NamedTemporaryFile(suffix='.xls') as tmp:
        for chunk in r.iter_content(chunk_size=1024):
            tmp.write(chunk)
            
        tmp.flush()

        df = pandas.read_excel(tmp.name,
                               sheetname='Data',
                               engine='xlrd',
                               skiprows=7,
                               skipfooter=11, # WARN: is this always 11?
                               index_col=0,
                               parse_cols="A:E,G:K")
        df.rename(columns={'Price' : 'Real Price',
                          'Dividend' : 'Real Dividend',
                          'Earnings' : 'Real Earnings'}, inplace=True)
        # Convert from Shiller's wonky date format to a real one
        df.index = pandas.DatetimeIndex([shiller_date_converter(n) for n in df.index])
    return df

shiller = get_shiller()
shiller.head()

,P,D,E,CPI,Rate GS10,Real Price,Real Dividend,Real Earnings,CAPE
1871-01-01,4.44,0.26,0.4,12.464061,5.320000,86.190133,5.047170,7.764877,NaN
1871-02-01,4.50,0.26,0.4,12.844641,5.323333,84.766585,4.897625,7.534808,NaN
1871-03-01,4.61,0.26,0.4,13.034972,5.326667,85.570679,4.826112,7.424788,NaN
1871-04-01,4.74,0.26,0.4,12.559226,5.330000,91.316573,5.008926,7.706040,NaN
1871-05-01,4.86,0.26,0.4,12.273812,5.333333,95.805617,5.125403,7.885236,NaN


In [4]:
def get_shiller_ch26(url='http://www.econ.yale.edu/~shiller/data/chapt26.xlsx'):
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    r = session.get(url, stream=True)

    with tempfile.NamedTemporaryFile(suffix='.xls') as tmp:
        for chunk in r.iter_content(chunk_size=1024):
            tmp.write(chunk)
            
        tmp.flush()

        df = pandas.read_excel(tmp.name,
                               sheetname='Data',
                               engine='xlrd',
                               skiprows=7,
                               skipfooter=12, # WARN: is this always 12?
                               index_col=0,
                               parse_cols="A,E")
        df.rename(columns={'Unnamed: 1': 'One-Year Interest Rate'}, inplace=True)
        df.index = pandas.DatetimeIndex([datetime.date(n, 1, 1) for n in df.index])
    return df

SHILLER_CHAPTER_26 = get_shiller_ch26()
SHILLER_CHAPTER_26.head()

,One-Year Interest Rate
1871-01-01,6.35
1872-01-01,7.81
1873-01-01,8.35
1874-01-01,6.86
1875-01-01,4.96


In [5]:
# Mix all of our data sources together!
all_by_month = pandas.concat([shiller, fred, SHILLER_CHAPTER_26], axis=1, join='outer')

In [6]:
# We only have annual interest rates from Shiller. We need to interpolate monthly rates...
all_by_month['One-Year Interest Rate'].interpolate(inplace=True)

In [7]:
def iterate_fund(ladder, yield_curve, max_maturity):
    ladder.reduce_maturities()
    ladder.generate_payments()
#    print(ladder.cash)
#    import pdb;pdb.set_trace()
    sold_bonds = ladder.sell_bonds(yield_curve)

    # Only buy a new bond if we actually sold one...
    if sold_bonds:
        ladder.buy_bond(yield_curve[max_maturity-1], max_maturity)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = ladder.get_nav(yield_curve)

    return (ladder, nav)

In [8]:
def a2m(annual_rate):
    return pow(annual_rate + 1, 1/12) - 1

class Bond:
    def __init__(self, face_value, yield_pct, maturity, payments_per_year=12):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        self.payments_per_year = payments_per_year
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))

    def gen_payment(self):
        return self.face_value * self.yield_pct / self.payments_per_year
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity / 12, (self.face_value * self.yield_pct), self.face_value)
        return -value
    
class BondLadder:
    def __init__(self, min_maturity, max_maturity):
        self.min_maturity = min_maturity
        self.max_maturity = max_maturity
        self.cash = 0
        
        self.ladder = set()
        
    def print_all(self):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond)
            
    def print_all_values(self, rates):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond.value(rates))
        
    def buy_bond(self, rate, maturity):
        b = Bond(self.cash, rate, maturity)
        self.add_bond(b)
        self.cash = 0
        return b
        
    def get_nav(self, rates):
        return self.cash + sum((b.value(rates) for b in self.ladder))

    def generate_payments(self):
        self.cash += sum((b.gen_payment() for b in self.ladder))        
        
    def __repr__(self):
        return ('%d-%d Ladder { Num Bonds: %d. }' % (self.max_maturity, self.min_maturity, len(self.ladder)))
        
    def add_bond(self, bond):
        #assert bond.maturity <= self.max_maturity
        #assert bond.maturity >= self.min_maturity
        self.ladder.add(bond)
    
    def reduce_maturities(self):
        for bond in self.ladder:
            bond.maturity -= 1

    def sell_bonds(self, rates):
        to_sell = filter(lambda bond: bond.maturity < self.min_maturity, self.ladder)
        to_sell = list(to_sell)
        self.ladder = self.ladder.difference(to_sell)
        self.cash += sum((b.value(rates) for b in to_sell))
        return to_sell

In [9]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]
    ladder = BondLadder(min_maturity, max_bonds)
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + a2m(bond_yield), i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.add_bond(b)
    return ladder
bootstrap([.0532]*120, 120, 108)

120-108 Ladder { Num Bonds: 13. }

In [10]:
def splice_data(raw_rates, series):
    # Start by loading the data we get from Shiller.
    # This will always exist.

    def safe_add(series_index, rate_index):
        # Don't overwrite any data we already have.
        if math.isnan(series.iloc[series_index]):
            series.iloc[series_index] = raw_rates[rate_index]

    safe_add(1 * 12 - 1, 'GS1')
    safe_add(1 * 12 - 1, 'One-Year Interest Rate')
    safe_add(10 * 12 - 1, 'Rate GS10')

    safe_add(2 * 12 - 1, 'GS2')
    safe_add(3 * 12 - 1, 'GS3')
    safe_add(5 * 12 - 1, 'GS5')
    safe_add(7 * 12 - 1, 'GS7')

    safe_add(0, 'TB4WK')
    safe_add(0, 'CD1M')

    safe_add(3 - 1, 'TB3MS')
    safe_add(3 - 1, 'M1329AUSM193NNBR')

    safe_add(6 - 1, 'TB6MS')

def build_yield_curve(raw_rates, yield_curve_size=10*12):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))

    # We use NaN to indicate "the data needs to be interpolated"
    # We have a few different data series that we splice together.
    splice_data(raw_rates, s)
    
    # This will do linear interpolation where it can.
    s.interpolate(inplace=True)
    
    # But it can still leave us with NaNs at the low end of the range
    s.fillna(method='backfill', inplace=True)
    
    # all of the data is in the form 3.71 but we want it to be .0371,
    # since that's what a percent actually is
    return s.apply(lambda x: x / 100).tolist()

In [11]:
['%.3f' % (s*100) for s in build_yield_curve(all_by_month.iloc[0])]

['6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.350',
 '6.340',
 '6.331',
 '6.321',
 '6.312',
 '6.302',
 '6.293',
 '6.283',
 '6.274',
 '6.264',
 '6.255',
 '6.245',
 '6.236',
 '6.226',
 '6.216',
 '6.207',
 '6.197',
 '6.188',
 '6.178',
 '6.169',
 '6.159',
 '6.150',
 '6.140',
 '6.131',
 '6.121',
 '6.112',
 '6.102',
 '6.092',
 '6.083',
 '6.073',
 '6.064',
 '6.054',
 '6.045',
 '6.035',
 '6.026',
 '6.016',
 '6.007',
 '5.997',
 '5.988',
 '5.978',
 '5.969',
 '5.959',
 '5.949',
 '5.940',
 '5.930',
 '5.921',
 '5.911',
 '5.902',
 '5.892',
 '5.883',
 '5.873',
 '5.864',
 '5.854',
 '5.845',
 '5.835',
 '5.825',
 '5.816',
 '5.806',
 '5.797',
 '5.787',
 '5.778',
 '5.768',
 '5.759',
 '5.749',
 '5.740',
 '5.730',
 '5.721',
 '5.711',
 '5.701',
 '5.692',
 '5.682',
 '5.673',
 '5.663',
 '5.654',
 '5.644',
 '5.635',
 '5.625',
 '5.616',
 '5.606',
 '5.597',
 '5.587',
 '5.578',
 '5.568',
 '5.558',
 '5.549',
 '5.539',
 '5.530',
 '5.520',
 '5.511',


In [12]:
bootstrap(build_yield_curve(all_by_month.iloc[-2]), 10 * 12, 4 * 12)

120-48 Ladder { Num Bonds: 73. }

In [13]:
def loop(ladder, rates, max_maturity):
    df = pandas.DataFrame(columns=['NAV', 'Change'])

    # The first iterations have fake data with duplicate years
    # But that's okay because we overwrite them with later data
    # (since they all have the same year)
    for (year, current_rates) in rates:
        if year.year % 5 == 0 and year.month == 1:
            print('Calculating...', year.year)
        (ladder, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Change' : None}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def make_annual_ladder(min_maturity, max_maturity, yields):
    rate = yields[max_maturity - 1]
    
    # Why is this -11 here?
    ladder = BondLadder(min_maturity - 11, max_maturity)

    face_value = 50
    for i in range(min_maturity, max_maturity + 1, 12):
        ladder.add_bond(Bond(face_value, rate, i))
        face_value = face_value * (1 + rate)

    return ladder

def simulate_monthly_turnover(min_maturiy, max_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = bootstrap(initial_yields, max_maturity, min_maturity)

    all_rates = itertools.chain(bootstrap_rates, rates.iterrows())

    return loop(ladder, all_rates, max_maturity)

def simulate_annual_turnover(min_maturity, max_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = make_annual_ladder(min_maturity, max_maturity, initial_yields)

    # longinvest actually simulates 1870 and assumes 1871 rates. That's why,
    # when the simulation starts in January 1871, all the bonds have paid 1 year
    # of interest and one of the bonds is ready to be sold.
    # So we need to generate 11 months of fake data to do the same simulation.
    # Why 11 months? The 12th month is the real January 1871 data.
    first_index = rates.head(1).index
    bootstrap_rates = itertools.repeat(next(rates.iterrows()), 11)
    all_rates = itertools.chain(bootstrap_rates, rates.iterrows())

    return loop(ladder, all_rates, max_maturity)

In [14]:
%%time
sim_results = simulate_annual_turnover(4, 10, all_by_month)
print(sim_results.head())

Calculating... 1875
Calculating... 1880
Calculating... 1885
Calculating... 1890
Calculating... 1895
Calculating... 1900
Calculating... 1905
Calculating... 1910
Calculating... 1915
Calculating... 1920
Calculating... 1925
Calculating... 1930
Calculating... 1935
Calculating... 1940
Calculating... 1945
Calculating... 1950
Calculating... 1955
Calculating... 1960
Calculating... 1965
Calculating... 1970
Calculating... 1975
Calculating... 1980
Calculating... 1985
Calculating... 1990
Calculating... 1995
Calculating... 2000
Calculating... 2005
Calculating... 2010
Calculating... 2015
                NAV      Change
1871-01-01  425.293  0.00216292
1871-02-01  426.213  0.00210301
1871-03-01  427.109  0.00204796
1871-04-01  427.984  0.00199774
1871-05-01  428.839  0.00195232
CPU times: user 12.3 s, sys: 175 ms, total: 12.5 s
Wall time: 13.4 s


In [15]:
sim_results.to_csv('bonds-monthly.csv')

In [16]:
def get_morningstar(secid):
    url = 'http://mschart.morningstar.com/chartweb/defaultChart?type=getcc&secids=%s&dataid=8225&startdate=1900-01-01&enddate=2016-11-18&currency=&format=1' % secid
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    # TODO: why doesn't this work!?!
    #r = session.get(url)
    r = requests.get(url)
    j = r.json()
    
    # The Morningstar data is pretty deeply nested....
    m = j['data']['r'][0]
    assert m['i'] == secid
    
    actual_data = m['t'][0]['d']
    # convert from strings to real data types
    as_dict = dict([(datetime.datetime.strptime(n['i'], '%Y-%m-%d'), float(n['v'])) for n in m['t'][0]['d']])
    
    # Strip out data?
    # Do we only want start of month, end of month, start of year, end of year, etc?
    s = pandas.Series(as_dict, name=secid)

    return s

barclays_index = get_morningstar('XIUSA000MJ')

# Use only final value for each calendar year
def annual(series):
    return series.groupby(by=lambda x: x.year).last()
# Use only final value for each calendar month
def monthly(series):
    return series.groupby(by=lambda x: datetime.date(x.year, x.month, 1)).last()

In [17]:
monthly_results = pandas.concat([sim_results, monthly(barclays_index)], axis=1, join='outer')

def calculate_change_prev(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        prev_val = df.iloc[i-1][column]
        change = (val - prev_val) / prev_val
        series.loc[df.iloc[i].name] = change
    return series

def calculate_change_next(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        next_val = df.iloc[i+1][column]
        change = (next_val - val) / val
        series.loc[df.iloc[i].name] = change
    return series

index_change = calculate_change_prev(monthly_results, 'XIUSA000MJ')
monthly_results = monthly_results.assign(index_change=index_change)
print(monthly_results[["Change", "index_change"]].corr())

              index_change
index_change           1.0


In [18]:
# This is correct when doing monthly calculations.
annual_results = monthly_results.groupby(by=lambda x: x.year).first()
s_nav = calculate_change_next(annual_results, 'NAV')
s_ind = calculate_change_next(annual_results, 'XIUSA000MJ')
annual_results = annual_results.assign(Change=s_nav, index_change=s_ind)
print(annual_results[["Change", "index_change"]].corr())
annual_results.to_csv('bonds-monthly-annual.csv')
print(annual_results.iloc[0])

                Change  index_change
Change        1.000000      0.904755
index_change  0.904755      1.000000
NAV             425.293282
Change            0.023298
XIUSA000MJ             NaN
index_change           NaN
Name: 1871, dtype: float64
